# Load Libraries

In [1]:
# packages
import os
import h5py
import time
import random
import itertools
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd
import numpy as np
from time import gmtime, strftime
import random
import keras.backend as K
from tensorflow import keras
from keras import layers
from keras import models
from keras import regularizers
from keras.models import model_from_json
from sklearn.utils import shuffle
from pandas import DataFrame

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from itertools import product
from sklearn.decomposition import PCA
# import tensorflow_addons as tfa

# EVALUATION modules
from scipy.stats import spearmanr

params = {'legend.fontsize': 20,
          'figure.figsize': (9,6),
         'axes.labelsize': 20,
         'axes.titlesize':20,
         'xtick.labelsize':'xx-large',
         'axes.linewidth' : 2,
         'ytick.labelsize':'xx-large'}

plt.rcParams.update(params)


# Helper Functions

## Preprocessing

In [2]:
def split_sequences(input_data, sequence_length, stride = 1, option = None):
    """
     
    """
    X = list()
    
    for i in range(0,len(input_data),stride):
        # find the end of this pattern
        end_ix = i + sequence_length
        
        # check if we are beyond the dataset
        if end_ix > len(input_data):
            break
        
        # gather input and output parts of the pattern
        if option=='last':
            seq_x = input_data[end_ix-1, :]
        elif option=='next':
            seq_x = input_data[end_ix, :]
        else:
            seq_x = input_data[i:end_ix, :]
        X.append(seq_x)
    
    return np.array(X)


## Plots

In [3]:
def plot_sensor_signal(x_real,x_pred,var_names,num=1000,figsize=10):
    input_dim = len(var_names)
    cols = min(np.floor(input_dim**0.5).astype(int),4)
    rows = (np.ceil(input_dim / cols)).astype(int)
    gs   = gridspec.GridSpec(rows, cols)
    fig  = plt.figure(figsize=(figsize, max(figsize, rows*2)))   
    for i in range(input_dim):
        ax = fig.add_subplot(gs[i])
        # ax.plot(np.arange(0,num),x_real[-num:,i],label="True",marker='.',markeredgewidth=0.25, markersize=8)
        # ax.plot(np.arange(0,num),x_pred[-num:,i],label="Pred",marker='.',markeredgewidth=0.25, markersize=8)
        ax.scatter(np.arange(0,num),x_real[-num:,i],label="True",marker='.')
        ax.scatter(np.arange(0,num),x_pred[-num:,i],label="Pred",marker='.')
        plt.title(var_names[i])
        ax.legend()
    plt.tight_layout()
    plt.show()

##  Sequence Generator

In [4]:
def sequence_generator(input_data, units, cycles, sequence_length=10,stride = 1, option=None):
    """
     # Generates dataset with windows of sequence_length      
    """  
    X = list()
    unit_num=[]
    c_num =[]
    for i, elem_u in enumerate(list(np.unique(units))):
        mask = np.ravel(units==elem_u)
        c_mask = cycles[mask]
        x_unit = input_data[mask]
        for j in np.unique(c_mask):
            mask = np.ravel(c_mask==j)
            seq_x_u = split_sequences(x_unit[mask],sequence_length, stride, option)
            X.append(seq_x_u)
            unit_num.extend(np.ones(len(seq_x_u),dtype = int)*elem_u)
            c_num.extend(np.ones(len(seq_x_u),dtype = int)*j)
    
    return np.vstack(X),np.array(unit_num).reshape(-1,1),np.array(c_num).reshape(-1,1)


def sequence_generator_per_unit(input_data, units, cycles, sequence_length=10, stride =1,option=None):
    """
     # Generates dataset with windows of sequence_length      
    """  
    X = list()
    unit_num=[]
    c_num =[]
    for i, elem_u in enumerate(list(np.unique(units))):
        mask = np.ravel(units==elem_u)
        x_unit = input_data[mask]
        seq_x_u = split_sequences(x_unit,sequence_length, stride, option)
        X.append(seq_x_u)
        unit_num.extend(np.ones(len(seq_x_u),dtype = int)*elem_u)
        c_num.append(split_sequences(cycles[mask],sequence_length, stride, option))
    
    return np.vstack(X),np.array(unit_num).reshape(-1,1),np.vstack(c_num)

# Models

# Model construction

## Load Data

In [5]:
# Flight Class FC
FC = 1

In [6]:
# Load data DEV
with h5py.File("FC"+str(FC)+"/FC"+str(FC)+'_dev'+".h5", 'r') as hdf:
            # Development set
            W_train = np.array(hdf.get('W_dev'), dtype='float16')             # W
            X_s_train = np.array(hdf.get('X_s_dev'), dtype='float16')         # X_s
            X_v_train = np.array(hdf.get('X_v_dev'), dtype='float16')         # X_v
            T_train = np.array(hdf.get('T_dev'), dtype='float16')             # T
            Y_train = np.array(hdf.get('Y_dev'), dtype='float16')             # RUL  
            A_train = np.array(hdf.get('A_dev'), dtype='float16')
            
            # Varnams
            W_var = np.array(hdf.get('W_var'))
            X_s_var = np.array(hdf.get('X_s_var'))  
            X_v_var = np.array(hdf.get('X_v_var')) 
            T_var = np.array(hdf.get('T_var'))
            A_var = np.array(hdf.get('A_var'))
            
                # from np.array to list dtype U4/U5
            W_var = list(np.array(W_var, dtype='U20'))
            X_s_var = list(np.array(X_s_var, dtype='U20'))  
            X_v_var = list(np.array(X_v_var, dtype='U20')) 
            T_var = list(np.array(T_var, dtype='U20'))
            A_var = list(np.array(A_var, dtype='U20'))
            
# Load data TEST
mode = '_test'
with h5py.File("FC"+str(FC)+"/FC"+str(FC)+'_test'+".h5", 'r') as hdf:
            # Development set
            W_test = np.array(hdf.get('W_test'), dtype='float16')             # W
            X_s_test = np.array(hdf.get('X_s_test'), dtype='float16')         # X_s
            X_v_test = np.array(hdf.get('X_v_test'), dtype='float16')         # X_v
            T_test = np.array(hdf.get('T_test'), dtype='float16')             # T
            Y_test = np.array(hdf.get('Y_test'), dtype='float16')             # RUL  
            A_test = np.array(hdf.get('A_test'), dtype='float16')

units_train=A_train[:,0].reshape(-1,1)
cycles_train=A_train[:,1].reshape(-1,1)
fc_train = A_train[:,2].reshape(-1,1)
hi_train = A_train[:,-1]

units_test=A_test[:,0].reshape(-1,1)
cycles_test=A_test[:,1].reshape(-1,1)
fc_test = A_test[:,2].reshape(-1,1)
hi_test = A_test[:,-1]

print("XS_train",X_s_train.shape)
print(units_train.shape)

XS_train (628969, 14)
(628969, 1)


### MaxMin Scale $X_s$ and $W$

In [7]:
# SCALE
# scaler_X = MinMaxScaler(feature_range=(-1,1))
scaler_X = MinMaxScaler()
X_s_train = scaler_X.fit_transform(X_s_train)
X_s_test = scaler_X.transform(X_s_test)

# scaler_W = MinMaxScaler(feature_range=(-1,1))
scaler_W = MinMaxScaler()
W_train = scaler_W.fit_transform(W_train)
W_test = scaler_W.transform(W_test)

scaler_Y = MinMaxScaler(feature_range=(0,1))
Y_train = scaler_Y.fit_transform(Y_train)
Y_test = scaler_Y.transform(Y_test)

### Downsamplig 0.1Hz

In [8]:
WINDOW_LEN = 50
stride = 1

#X_windows_test, U_windows_test,C_windows_test=sequence_generator(X_s_test,units_test,cycles_test,sequence_length=WINDOW_LEN,stride = stride)
#W_windows_test,_,_=sequence_generator(W_test,units_test,cycles_test,sequence_length=WINDOW_LEN,stride = stride)
#Y_windows_test,_,_=sequence_generator(Y_test,units_test,cycles_test,sequence_length=WINDOW_LEN,option='last',stride = stride)


In [9]:
X_windows_test, U_windows_test,C_windows_test=sequence_generator_per_unit(X_s_test,units_test,cycles_test,sequence_length=WINDOW_LEN,stride = stride)
W_windows_test,_,_=sequence_generator_per_unit(W_test,units_test,cycles_test,sequence_length=WINDOW_LEN,stride = stride)
Y_windows_test,_,_=sequence_generator_per_unit(Y_test,units_test,cycles_test,sequence_length=WINDOW_LEN,option='last',stride = stride)

MemoryError: Unable to allocate 20.8 MiB for an array with shape (15591, 50, 14) and data type float16

## RUL MODEL

In [ ]:
#seed = 5
seed=229
np.random.seed(seed)

In [ ]:
#model.load_weights("model.h5")

json_file = open("RUL_MODEL/FC"+str(FC)+"/"+"/model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
rul_model = model_from_json(loaded_model_json)
rul_model.load_weights("RUL_MODEL/FC"+str(FC)+"/"+"/model.h5")

### Test

In [ ]:
rul_predicted = rul_model.predict((X_windows_test,W_windows_test))

In [ ]:
rul_predicted_ = scaler_Y.inverse_transform(rul_predicted)
groud_truth = scaler_Y.inverse_transform(Y_windows_test)

In [ ]:
rul_predicted

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math 
mae=mean_absolute_error(rul_predicted_,groud_truth)
mse=mean_squared_error(rul_predicted_,groud_truth)
rmse=np.sqrt(mse)

print("MAE:",mae)
print("MSE:",mse)
print("RMSE:",rmse)

# Plots

## Remaining Useful Life Estimation

In [ ]:
df_U_test = DataFrame(data=U_windows_test, columns=['unit']) 
df_Y_test = DataFrame(data=Y_windows_test, columns=['RUL']) 
df_Y_hat_test = DataFrame(data=rul_predicted, columns=['RUL_hat'])
df_test = pd.concat([df_U_test, df_Y_test,df_Y_hat_test], axis=1)

In [ ]:
df_test

In [ ]:
list(np.unique(units_test))

In [ ]:
C_windows_test